In [1]:
%load_ext autoreload
%autoreload 1
%aimport rails,aise

In [2]:
import torch
import torch.nn as nn
from collections import deque
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import model as _model
import utils as _tools

from resnet import ResNet18

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
"""
initialize the trainned model
"""

resnet = _model.TransferModel(use_cpu=False)

resnet.model.load_state_dict(torch.load("models/Cardiomegaly_resnet18.pth"))
resnet.model.to(DEVICE)
resnet.model.eval()

print("done.")

AttributeError: 'str' object has no attribute 'copy'

In [14]:
# model.feature.load_state_dict(nn.Sequential(
#     resnet.model.model.conv1,
#     resnet.model.model.bn1,
#     resnet.model.model.relu,
#     resnet.model.model.maxpool
# ).state_dict())
# model.layer1.load_state_dict(resnet.model.model.layer1.state_dict())
# model.layer2.load_state_dict(resnet.model.model.layer2.state_dict())
# model.layer3.load_state_dict(resnet.model.model.layer3.state_dict())
# model.layer4.load_state_dict(resnet.model.model.layer4.state_dict())
# model.classifier.load_state_dict(nn.Sequential(
#     resnet.model.model.avgpool,
#     nn.Flatten(start_dim=1),
#     resnet.model.model.fc,
# ).state_dict())

<All keys matched successfully>

In [15]:
(model(x_test.to(DEVICE)).max(dim=1)[1]==y_test.to(DEVICE)).float().mean()

tensor(0.8200, device='cuda:0')

In [10]:
model = ResNet18(num_classes=2)
model.load_state_dict(torch.load("./models/chexpert_resnet18.pt"))
model.to(DEVICE)
model.eval()

ResNet(
  (feature): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu):

In [11]:
# the training data from the data loader
n_rows = 1000 # how many rows to sample
row_count = 0
x = []
y = []
for image, labels in resnet.dataloader_train.dataset:
    row_count += 1
    x.append(image)
    y.append(labels[0]) # the label for cadiomegaly
    
    if row_count == n_rows:
        break
    
x_train = torch.FloatTensor(x)
y_train = torch.LongTensor(y)

print("done")

done


In [6]:
# the testing data from the data loader
n_rows = 100 # how many rows to sample
row_count = 0
x = []
y = []
for image, labels in resnet.dataloader_valid.dataset:
    row_count += 1
    x.append(image)
    y.append(labels[0]) # the label for cadiomegaly
    
    if row_count == n_rows:
        break
    
x_test = torch.FloatTensor(x)
y_test = torch.LongTensor(y)

print("done")

done


In [33]:
CONFIG = {
    "start_layer": 1,
    "n_class": 2,
    "aise_params": [
        {"hidden_layer": 2, 
         "sampling_temperature": 1, 
         "max_generation": 10, 
         "n_neighbors" : 3,
         "n_population" : 2 * 3,
         "mut_range": (.005, .015)},
    ]
}

rails_clf = rails.RAILS(model, 
              CONFIG, 
              x_train,
              y_train)

1


In [41]:
(model(x_test.to(DEVICE)).max(dim=1)[1]==y_test.to(DEVICE)).float().mean()

tensor(0.3200, device='cuda:0')

In [38]:
y_pred = rails_clf.predict(x_test).argmax(axis=1)
(y_pred==y_test.numpy()).astype("float").mean()

0.81